# SYDE 556/750: Simulating Neurobiological Systems

## Memory

- We've seen how to represent symbol-like structures using vectors
- Typically high dimensional vectors
- How can we store those over short periods of time (working memory)
- Long periods of time?

### Remembering a Plan

- Previously, we talked about a model of the Tower of Hanoi task
- Move disks around from one configuration to another
- A common situation:
    - Trying to move disk 3 to peg A, but disk 2 is in the way, so we need to move disk 2 to peg B, but disk 1 is in the way, so we move disk 1 to peg C
    - When we do that, it'd be nice to be able to remember the previous steps in the chain of reasoning
    - So we could go back to trying to move disk 2 to peg B, rather than going all the way back to the beginning
    - Timing data indicates people do this
- What do we need to store?    


- Need to remember disks and what peg they go to
    - can't do `D3 + A  + D2 + B`
    - since that's the same as `D3 + B  + D2 + A`
- Something like: `D3` $\circledast$ `A + D2` $\circledast$ `B` 
- Associative Memory

- Let's start by just computing the convolution

In [1]:
D = 64
subdim = 8
N = 500

import nef
net=nef.Network('Symbols', fixed_seed=1, quick=True) #Create the network object

net.make('A',neurons=1,dimensions=D,mode='direct')  # don't bother simulating these neurons
net.make('B',neurons=1,dimensions=D,mode='direct')  # don't bother simulating these neurons

net.make_array('C',N,D/subdim,dimensions=subdim,radius=1.0/math.sqrt(D), seed=2) 

conv = nef.convolution.make_convolution(net,'*','A','B','C',200, seed=3)

net.add_to_nengo()


ImportError: No module named nef

- Now let's extract out a desired answer

In [ ]:
D = 64
subdim = 8
N = 500

import nef
net=nef.Network('Symbols', fixed_seed=1, quick=True) #Create the network object

net.make('A',1,D,mode='direct')
net.make('B',1,D,mode='direct')
net.make_array('C',N,D/subdim,dimensions=subdim,radius=1.0/math.sqrt(D), seed=2) 

conv = nef.convolution.make_convolution(net,'*','A','B','C',200, seed=3)


net.make('E',1,D,mode='direct')
net.make('F',1,D,mode='direct')

conv = nef.convolution.make_convolution(net,'/','C','E','F',200, invert_second=True, seed=3)


net.add_to_nengo()



- But that's not much good without a memory
- How do we add one?


In [ ]:
D = 64
subdim = 8
N = 500

import nef
net=nef.Network('Symbols', fixed_seed=1, quick=True) #Create the network object

net.make('A',1,D,mode='direct')
net.make('B',1,D,mode='direct')
net.make_array('C',N,D/subdim,dimensions=subdim,radius=1.0/math.sqrt(D), seed=2) 

net.connect('C', 'C', pstc=0.1)

conv = nef.convolution.make_convolution(net,'*','A','B','C',200, seed=3)


net.make('E',1,D,mode='direct')
net.make('F',1,D,mode='direct')

conv = nef.convolution.make_convolution(net,'/','C','E','F',200, invert_second=True, seed=3)


net.add_to_nengo()


- Things to note
    - Memory slowly decays
    - If you push in a new pair for too long, it can wipe out the old pair(s)
        - Note that this relies on the saturation behaviour of NEF networks
        - Kind of like implicit normalization
    - Memory capacity increases with dimensionality
        - Also dependent on the number of different possible items in memory (vocabulary size)
    - 512 dimensions is suffienct to store ~8 pairs, with a vocabulary size of 100,000 terms
        - Note that this is what's needed for storing simple sentences